# Weather Model Training

This notebook trains a Random Forest model to predict weather conditions based on historical data from Semarang (2010-2025).
The model output is designed to be compatible with the `weatherapp_backend` application.

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Ensure plots are displayed inline
%matplotlib inline

# Set style for better visualizations
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## 1. Load Data

In [2]:
csv_path = '../data_collections/semarang_weather_2010_2025.csv'
df = pd.read_csv(csv_path)

# Display first few rows
print(f"Dataset shape: {df.shape}")
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '../data_collections/semarang_weather_2010_2025.csv'

In [ ]:
# Check data types and missing values
df.info()

## 2. Preprocessing

The raw data is hourly (or similar). The application expects daily predictions.
We need to resample the data to a daily frequency and aggregate the values.

**Aggregation Logic:**
*   `temperature`: Max, Min, Mean
*   `humidity`: Mean
*   `wind_speed`: Mean
*   `pressure`: Mean
*   `rain`: Sum

**Target Variable `conditions`:**
The backend expects a `conditions` integer mapping:
*   0: 'Clear'
*   1: 'Overcast'
*   2: 'Partially cloudy'
*   3: 'Rain'
*   4: 'Rain, Overcast'
*   5: 'Rain, Partially cloudy'

Since our dataset lacks cloud cover, we will approximate:
*   If `rain` > 0: `3` (Rain)
*   Else: `0` (Clear)

In [ ]:
# Convert time to datetime
df['time'] = pd.to_datetime(df['time'])
df.set_index('time', inplace=True)

# Resample to daily
daily_df = df.resample('D').agg({
    'temperature': ['max', 'min', 'mean'],
    'humidity': 'mean',
    'wind_speed': 'mean',
    'pressure': 'mean',
    'rain': 'sum'
})

# Flatten MultiIndex columns
daily_df.columns = ['tempmax', 'tempmin', 'temp', 'humidity', 'windspeed', 'pressure', 'rain_sum']

# Drop rows with NaN values (if any)
daily_df.dropna(inplace=True)

# Create 'conditions' target
daily_df['conditions'] = daily_df['rain_sum'].apply(lambda x: 3 if x > 0 else 0)

print(f"Daily data shape: {daily_df.shape}")
daily_df.head()

## 3. Feature Engineering

The model input in `main.py` is `[day, month, year]`.

In [ ]:
daily_df['day'] = daily_df.index.day
daily_df['month'] = daily_df.index.month
daily_df['year'] = daily_df.index.year

# Features (X) and Targets (y)
X = daily_df[['day', 'month', 'year']]
y = daily_df[['tempmax', 'tempmin', 'temp', 'humidity', 'windspeed', 'pressure', 'conditions']]

print("X shape:", X.shape)
print("y shape:", y.shape)

## 4. Train-Test Split

Split data into training (80%) and testing (20%) sets to evaluate model performance.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set size: {X_train.shape[0]} samples")
print(f"Testing set size: {X_test.shape[0]} samples")

## 5. Model Training

Using `RandomForestRegressor` as it supports multi-output regression natively.

In [ ]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)

print("Model trained successfully.")

## 6. Model Evaluation

Evaluate model performance on test set with multiple metrics.

In [ ]:
# Make predictions on test set
y_pred = rf_model.predict(X_test)

# Calculate metrics for each target
target_names = ['tempmax', 'tempmin', 'temp', 'humidity', 'windspeed', 'pressure', 'conditions']

print("="*80)
print("Model Performance Metrics")
print("="*80)

for i, name in enumerate(target_names):
    mae = mean_absolute_error(y_test.iloc[:, i], y_pred[:, i])
    rmse = np.sqrt(mean_squared_error(y_test.iloc[:, i], y_pred[:, i]))
    r2 = r2_score(y_test.iloc[:, i], y_pred[:, i])
    
    print(f"\n{name.upper()}:")
    print(f"  MAE  (Mean Absolute Error): {mae:.4f}")
    print(f"  RMSE (Root Mean Squared Error): {rmse:.4f}")
    print(f"  R²   (Coefficient of Determination): {r2:.4f}")

## 7. Visualization - Actual vs Predicted

Compare actual and predicted values for each target variable.

In [ ]:
fig, axes = plt.subplots(3, 3, figsize=(18, 15))
axes = axes.flatten()

for i, name in enumerate(target_names):
    ax = axes[i]
    
    # Scatter plot
    ax.scatter(y_test.iloc[:, i], y_pred[:, i], alpha=0.3, s=10)
    
    # Perfect prediction line
    min_val = min(y_test.iloc[:, i].min(), y_pred[:, i].min())
    max_val = max(y_test.iloc[:, i].max(), y_pred[:, i].max())
    ax.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, label='Perfect Prediction')
    
    # Labels and title
    ax.set_xlabel('Actual Values', fontsize=10)
    ax.set_ylabel('Predicted Values', fontsize=10)
    ax.set_title(f'{name.upper()}', fontsize=12, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Calculate R²
    r2 = r2_score(y_test.iloc[:, i], y_pred[:, i])
    ax.text(0.05, 0.95, f'R² = {r2:.4f}', transform=ax.transAxes, 
            verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# Remove extra subplot
fig.delaxes(axes[7])
fig.delaxes(axes[8])

plt.tight_layout()
plt.savefig('../outputs/actual_vs_predicted.png', dpi=150, bbox_inches='tight')
plt.show()

print("Visualization saved as '../outputs/actual_vs_predicted.png'")

## 8. Residual Analysis

Analyze residuals (errors) to understand model performance.

In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(20, 8))
axes = axes.flatten()

for i, name in enumerate(target_names):
    ax = axes[i]
    
    # Calculate residuals
    residuals = y_test.iloc[:, i] - y_pred[:, i]
    
    # Histogram of residuals
    ax.hist(residuals, bins=50, edgecolor='black', alpha=0.7)
    ax.axvline(x=0, color='red', linestyle='--', linewidth=2, label='Zero Error')
    
    ax.set_xlabel('Residual (Actual - Predicted)', fontsize=10)
    ax.set_ylabel('Frequency', fontsize=10)
    ax.set_title(f'Residuals - {name.upper()}', fontsize=12, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Add mean and std
    mean_res = residuals.mean()
    std_res = residuals.std()
    ax.text(0.05, 0.95, f'Mean: {mean_res:.4f}\nStd: {std_res:.4f}', 
            transform=ax.transAxes, verticalalignment='top', 
            bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.5))

# Remove extra subplot
fig.delaxes(axes[7])

plt.tight_layout()
plt.savefig('../outputs/residual_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print("Residual analysis saved as '../outputs/residual_analysis.png'")

## 9. Feature Importance

Analyze which input features (day, month, year) are most important for predictions.

In [ ]:
# Get feature importances
feature_names = ['Day', 'Month', 'Year']
importances = rf_model.feature_importances_

# Create bar plot
plt.figure(figsize=(10, 6))
bars = plt.bar(feature_names, importances, color=['#FF6B6B', '#4ECDC4', '#45B7D1'])

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.4f}',
             ha='center', va='bottom', fontsize=12, fontweight='bold')

plt.xlabel('Features', fontsize=12, fontweight='bold')
plt.ylabel('Importance', fontsize=12, fontweight='bold')
plt.title('Feature Importance in Weather Prediction Model', fontsize=14, fontweight='bold')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig('../outputs/feature_importance.png', dpi=150, bbox_inches='tight')
plt.show()

print("Feature importance saved as '../outputs/feature_importance.png'")

## 10. Time Series Comparison

Visualize predictions vs actual values over time (sample of 100 days).

In [ ]:
# Get a sample of test data (last 100 days)
sample_size = min(100, len(X_test))
sample_indices = sorted(y_test.index[-sample_size:])

# Get predictions for sample
y_test_sample = y_test.loc[sample_indices]
X_test_sample = X_test.loc[sample_indices]
y_pred_sample = rf_model.predict(X_test_sample)

# Plot temperature comparison
fig, axes = plt.subplots(3, 1, figsize=(16, 12))

# Temperature
axes[0].plot(sample_indices, y_test_sample['temp'], label='Actual', marker='o', markersize=3, linewidth=2)
axes[0].plot(sample_indices, y_pred_sample[:, 2], label='Predicted', marker='s', markersize=3, linewidth=2, alpha=0.7)
axes[0].set_xlabel('Date', fontsize=11)
axes[0].set_ylabel('Temperature (°C)', fontsize=11)
axes[0].set_title('Temperature - Actual vs Predicted', fontsize=13, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Humidity
axes[1].plot(sample_indices, y_test_sample['humidity'], label='Actual', marker='o', markersize=3, linewidth=2)
axes[1].plot(sample_indices, y_pred_sample[:, 3], label='Predicted', marker='s', markersize=3, linewidth=2, alpha=0.7)
axes[1].set_xlabel('Date', fontsize=11)
axes[1].set_ylabel('Humidity (%)', fontsize=11)
axes[1].set_title('Humidity - Actual vs Predicted', fontsize=13, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Wind Speed
axes[2].plot(sample_indices, y_test_sample['windspeed'], label='Actual', marker='o', markersize=3, linewidth=2)
axes[2].plot(sample_indices, y_pred_sample[:, 4], label='Predicted', marker='s', markersize=3, linewidth=2, alpha=0.7)
axes[2].set_xlabel('Date', fontsize=11)
axes[2].set_ylabel('Wind Speed (m/s)', fontsize=11)
axes[2].set_title('Wind Speed - Actual vs Predicted', fontsize=13, fontweight='bold')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../outputs/time_series_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("Time series comparison saved as '../outputs/time_series_comparison.png'")

## 11. Model Saving

Save the final model trained on ALL data (for production use).

In [ ]:
# Retrain on full dataset for production
print("Retraining model on full dataset for production...")
rf_model_final = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf_model_final.fit(X, y)

# Save model
with open('../models/rf_model_pkl', 'wb') as f:
    pickle.dump(rf_model_final, f)

print("✓ Model saved to rf_model_pkl")
print("\n" + "="*80)
print("NEXT STEPS:")
print("="*80)
print("1. Copy '../models/rf_model_pkl' to '../weatherapp_backend/' directory")
print("2. The model is now ready to use with main.py")
print("="*80)

## 12. Verification

Load the model and test with a sample input to ensure compatibility with `main.py`.

In [ ]:
# Load model
with open('../models/rf_model_pkl', 'rb') as f:
    loaded_model = pickle.load(f)

# Test prediction for a specific date (e.g., 27-11-2025)
# Using DataFrame to avoid feature name warning
test_input_df = pd.DataFrame([[27, 11, 2025]], columns=['day', 'month', 'year'])
prediction = loaded_model.predict(test_input_df)

print("="*80)
print("MODEL VERIFICATION")
print("="*80)
print(f"Test Input: Day={test_input_df.iloc[0]['day']}, Month={test_input_df.iloc[0]['month']}, Year={test_input_df.iloc[0]['year']}")
print("\nPrediction shape:", prediction.shape)
print("\nPredicted Weather Conditions:")
print("-" * 80)

# Expected output columns: tempmax, tempmin, temp, humidity, windspeed, pressure, conditions
columns = ['tempmax', 'tempmin', 'temp', 'humidity', 'windspeed', 'pressure', 'conditions']
conditions_map = {0: 'Clear', 1: 'Overcast', 2: 'Partially cloudy', 3: 'Rain', 
                  4: 'Rain, Overcast', 5: 'Rain, Partially cloudy'}

for col, val in zip(columns, prediction[0]):
    if col == 'conditions':
        cond_int = int(round(val))
        print(f"  {col.upper():<15}: {cond_int} ({conditions_map.get(cond_int, 'Unknown')})")
    else:
        print(f"  {col.upper():<15}: {val:.4f}")

print("="*80)
print("✓ Model verification complete! Model is compatible with main.py")
print("="*80)

## 13. 7-Day Weather Forecast

Predict weather conditions for the next 7 days from today (27 November 2025).

In [ ]:
# Starting date (today)
start_date = datetime(2025, 11, 27)

print("="*100)
print("7-DAY WEATHER FORECAST FROM TODAY")
print("="*100)
print(f"Starting Date: {start_date.strftime('%A, %d %B %Y')}\n")

# Store predictions for visualization
forecast_data = []

conditions_map = {0: 'Clear ☀️', 1: 'Overcast ☁️', 2: 'Partially cloudy ⛅', 3: 'Rain 🌧️', 
                  4: 'Rain, Overcast 🌧️☁️', 5: 'Rain, Partially cloudy 🌦️'}

for i in range(7):
    current_date = start_date + timedelta(days=i)
    day = current_date.day
    month = current_date.month
    year = current_date.year
    
    # Create DataFrame input to avoid warning
    input_df = pd.DataFrame([[day, month, year]], columns=['day', 'month', 'year'])
    
    # Predict
    pred = loaded_model.predict(input_df)[0]
    
    # Store for later visualization
    forecast_data.append({
        'date': current_date,
        'date_str': current_date.strftime('%A, %d %b'),
        'tempmax': pred[0],
        'tempmin': pred[1],
        'temp': pred[2],
        'humidity': pred[3],
        'windspeed': pred[4],
        'pressure': pred[5],
        'conditions': int(round(pred[6]))
    })
    
    # Print daily forecast
    cond_int = int(round(pred[6]))
    print(f"📅 Day {i+1}: {current_date.strftime('%A, %d %B %Y')}")
    print("-" * 100)
    print(f"   🌡️  Temperature: {pred[2]:.1f}°C  (Min: {pred[1]:.1f}°C | Max: {pred[0]:.1f}°C)")
    print(f"   💧 Humidity: {pred[3]:.1f}%")
    print(f"   💨 Wind Speed: {pred[4]:.1f} m/s")
    print(f"   ⏲️  Pressure: {pred[5]:.1f} hPa")
    print(f"   🌤️  Conditions: {conditions_map.get(cond_int, 'Unknown')}")
    print()

print("="*100)

### Visualize 7-Day Forecast

In [ ]:
# Convert to DataFrame
forecast_df = pd.DataFrame(forecast_data)

# Create comprehensive forecast visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Temperature forecast
ax1 = axes[0, 0]
ax1.plot(forecast_df['date_str'], forecast_df['tempmax'], marker='o', linewidth=2.5, 
         label='Max Temp', color='#FF6B6B', markersize=8)
ax1.plot(forecast_df['date_str'], forecast_df['temp'], marker='s', linewidth=2.5, 
         label='Avg Temp', color='#FFA500', markersize=8)
ax1.plot(forecast_df['date_str'], forecast_df['tempmin'], marker='^', linewidth=2.5, 
         label='Min Temp', color='#4ECDC4', markersize=8)
ax1.set_xlabel('Date', fontsize=11, fontweight='bold')
ax1.set_ylabel('Temperature (°C)', fontsize=11, fontweight='bold')
ax1.set_title('📈 7-Day Temperature Forecast', fontsize=13, fontweight='bold')
ax1.legend(loc='best')
ax1.grid(True, alpha=0.3)
ax1.tick_params(axis='x', rotation=45)

# Humidity forecast
ax2 = axes[0, 1]
bars = ax2.bar(forecast_df['date_str'], forecast_df['humidity'], 
               color='#45B7D1', edgecolor='black', linewidth=1.5)
ax2.set_xlabel('Date', fontsize=11, fontweight='bold')
ax2.set_ylabel('Humidity (%)', fontsize=11, fontweight='bold')
ax2.set_title('💧 7-Day Humidity Forecast', fontsize=13, fontweight='bold')
ax2.grid(axis='y', alpha=0.3)
ax2.tick_params(axis='x', rotation=45)
# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.1f}%', ha='center', va='bottom', fontsize=9)

# Wind speed forecast
ax3 = axes[1, 0]
ax3.plot(forecast_df['date_str'], forecast_df['windspeed'], marker='D', linewidth=2.5, 
         color='#95E1D3', markersize=8)
ax3.fill_between(range(len(forecast_df)), forecast_df['windspeed'], alpha=0.3, color='#95E1D3')
ax3.set_xlabel('Date', fontsize=11, fontweight='bold')
ax3.set_ylabel('Wind Speed (m/s)', fontsize=11, fontweight='bold')
ax3.set_title('💨 7-Day Wind Speed Forecast', fontsize=13, fontweight='bold')
ax3.grid(True, alpha=0.3)
ax3.set_xticks(range(len(forecast_df)))
ax3.set_xticklabels(forecast_df['date_str'], rotation=45)

# Weather conditions forecast
ax4 = axes[1, 1]
conditions_colors = {0: '#FFD93D', 1: '#A8A8A8', 2: '#C0C0C0', 
                     3: '#6C5CE7', 4: '#4A69BD', 5: '#5F9EA0'}
colors = [conditions_colors.get(c, '#CCCCCC') for c in forecast_df['conditions']]
bars = ax4.bar(forecast_df['date_str'], forecast_df['conditions'], 
               color=colors, edgecolor='black', linewidth=1.5)
ax4.set_xlabel('Date', fontsize=11, fontweight='bold')
ax4.set_ylabel('Condition Code', fontsize=11, fontweight='bold')
ax4.set_title('🌤️ 7-Day Weather Conditions', fontsize=13, fontweight='bold')
ax4.set_yticks([0, 1, 2, 3, 4, 5])
ax4.set_yticklabels(['Clear', 'Overcast', 'Partly Cloudy', 'Rain', 'Rain+Overcast', 'Rain+Partly'])
ax4.grid(axis='y', alpha=0.3)
ax4.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('../outputs/7day_forecast.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ 7-Day forecast visualization saved as '../outputs/7day_forecast.png'")

## 14. Hourly Weather Forecast for 3 Days

Generate hourly predictions for the next 3 days (72 hours) using realistic diurnal patterns.

In [ ]:
# Generate hourly forecast for 3 days (72 hours)
start_date = datetime(2025, 11, 27, 0, 0)  # Start from midnight
hours_to_predict = 72  # 3 days

print("="*100)
print("HOURLY WEATHER FORECAST FOR 3 DAYS (72 HOURS)")
print("="*100)
print(f"Starting: {start_date.strftime('%A, %d %B %Y %H:%M')}\n")

hourly_forecast = []

# Get daily predictions first
daily_predictions = {}
for day_offset in range(3):
    current_date = start_date + timedelta(days=day_offset)
    input_df = pd.DataFrame([[current_date.day, current_date.month, current_date.year]], 
                           columns=['day', 'month', 'year'])
    pred = loaded_model.predict(input_df)[0]
    daily_predictions[day_offset] = {
        'tempmax': pred[0],
        'tempmin': pred[1],
        'temp_avg': pred[2],
        'humidity': pred[3],
        'windspeed': pred[4],
        'pressure': pred[5],
        'conditions': int(round(pred[6]))
    }

# Generate hourly data with realistic patterns
for hour in range(hours_to_predict):
    current_time = start_date + timedelta(hours=hour)
    hour_of_day = current_time.hour
    day_offset = hour // 24
    
    daily_pred = daily_predictions[day_offset]
    
    # Temperature pattern: cooler at night (2am), warmest at 2pm
    # Using sinusoidal pattern
    hour_angle = (hour_of_day - 2) * (2 * np.pi / 24)  # 2am is minimum
    temp_factor = (np.sin(hour_angle) + 1) / 2  # 0 to 1
    temp = daily_pred['tempmin'] + (daily_pred['tempmax'] - daily_pred['tempmin']) * temp_factor
    
    # Humidity: inverse to temperature (higher at night)
    humidity_base = daily_pred['humidity']
    humidity_variation = 10  # ±10%
    humidity = humidity_base + humidity_variation * (1 - temp_factor)
    humidity = min(100, max(0, humidity))  # Clamp to 0-100
    
    # Wind speed: slightly higher during day
    wind_factor = 0.7 + 0.3 * temp_factor  # 0.7 to 1.0
    windspeed = daily_pred['windspeed'] * wind_factor + np.random.normal(0, 0.3)
    windspeed = max(0, windspeed)
    
    # Pressure: relatively stable with small variations
    pressure = daily_pred['pressure'] + np.random.normal(0, 0.5)
    
    hourly_forecast.append({
        'datetime': current_time,
        'datetime_str': current_time.strftime('%Y-%m-%d %H:%M'),
        'day': current_time.strftime('%d %b'),
        'hour': hour_of_day,
        'temp': temp,
        'humidity': humidity,
        'windspeed': windspeed,
        'pressure': pressure,
        'conditions': daily_pred['conditions']
    })

# Convert to DataFrame
hourly_df = pd.DataFrame(hourly_forecast)

# Display summary (first 24 hours)
print("Sample: First 24 Hours Forecast")
print("=" * 100)
for i in range(24):
    data = hourly_df.iloc[i]
    if i % 3 == 0:  # Print every 3 hours for readability
        conditions_name = {0: '☀️ Clear', 1: '☁️ Overcast', 2: '⛅ Partly Cloudy', 3: '🌧️ Rain'}[data['conditions']]
        print(f"{data['datetime_str']} | Temp: {data['temp']:.1f}°C | Humidity: {data['humidity']:.1f}% | "
              f"Wind: {data['windspeed']:.1f} m/s | {conditions_name}")

print("\n" + "=" * 100)
print(f"Total hourly predictions: {len(hourly_df)} hours")
print("Full data saved in 'hourly_df' variable")
print("=" * 100)

### Visualize 72-Hour Hourly Forecast

In [ ]:
# Create comprehensive hourly visualization
fig, axes = plt.subplots(4, 1, figsize=(18, 14))

hours_range = range(len(hourly_df))
hour_labels = [hourly_df.iloc[i]['datetime_str'] if i % 6 == 0 else '' for i in hours_range]

# Temperature
ax1 = axes[0]
ax1.plot(hours_range, hourly_df['temp'], linewidth=2, color='#FF6B6B', label='Temperature')
ax1.fill_between(hours_range, hourly_df['temp'], alpha=0.3, color='#FF6B6B')
ax1.set_ylabel('Temperature (°C)', fontsize=12, fontweight='bold')
ax1.set_title('🌡️ 72-Hour Temperature Forecast', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.legend(loc='upper right')
ax1.set_xticks(range(0, len(hourly_df), 6))
ax1.set_xticklabels([hourly_df.iloc[i]['datetime_str'] for i in range(0, len(hourly_df), 6)], 
                     rotation=45, ha='right')

# Add day separators
for day in range(1, 4):
    ax1.axvline(x=day*24, color='gray', linestyle='--', alpha=0.5, linewidth=2)

# Humidity
ax2 = axes[1]
ax2.plot(hours_range, hourly_df['humidity'], linewidth=2, color='#45B7D1', label='Humidity')
ax2.fill_between(hours_range, hourly_df['humidity'], alpha=0.3, color='#45B7D1')
ax2.set_ylabel('Humidity (%)', fontsize=12, fontweight='bold')
ax2.set_title('💧 72-Hour Humidity Forecast', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.legend(loc='upper right')
ax2.set_xticks(range(0, len(hourly_df), 6))
ax2.set_xticklabels([hourly_df.iloc[i]['datetime_str'] for i in range(0, len(hourly_df), 6)], 
                     rotation=45, ha='right')

for day in range(1, 4):
    ax2.axvline(x=day*24, color='gray', linestyle='--', alpha=0.5, linewidth=2)

# Wind Speed
ax3 = axes[2]
ax3.plot(hours_range, hourly_df['windspeed'], linewidth=2, color='#95E1D3', label='Wind Speed')
ax3.fill_between(hours_range, hourly_df['windspeed'], alpha=0.3, color='#95E1D3')
ax3.set_ylabel('Wind Speed (m/s)', fontsize=12, fontweight='bold')
ax3.set_title('💨 72-Hour Wind Speed Forecast', fontsize=14, fontweight='bold')
ax3.grid(True, alpha=0.3)
ax3.legend(loc='upper right')
ax3.set_xticks(range(0, len(hourly_df), 6))
ax3.set_xticklabels([hourly_df.iloc[i]['datetime_str'] for i in range(0, len(hourly_df), 6)], 
                     rotation=45, ha='right')

for day in range(1, 4):
    ax3.axvline(x=day*24, color='gray', linestyle='--', alpha=0.5, linewidth=2)

# Pressure
ax4 = axes[3]
ax4.plot(hours_range, hourly_df['pressure'], linewidth=2, color='#A8E6CF', label='Pressure')
ax4.fill_between(hours_range, hourly_df['pressure'], alpha=0.3, color='#A8E6CF')
ax4.set_ylabel('Pressure (hPa)', fontsize=12, fontweight='bold')
ax4.set_xlabel('Date & Time', fontsize=12, fontweight='bold')
ax4.set_title('⏲️ 72-Hour Pressure Forecast', fontsize=14, fontweight='bold')
ax4.grid(True, alpha=0.3)
ax4.legend(loc='upper right')
ax4.set_xticks(range(0, len(hourly_df), 6))
ax4.set_xticklabels([hourly_df.iloc[i]['datetime_str'] for i in range(0, len(hourly_df), 6)], 
                     rotation=45, ha='right')

for day in range(1, 4):
    ax4.axvline(x=day*24, color='gray', linestyle='--', alpha=0.5, linewidth=2)

plt.tight_layout()
plt.savefig('../outputs/72hour_forecast.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ 72-hour hourly forecast visualization saved as '../outputs/72hour_forecast.png'")
print("\n" + "="*100)
print("EXPORT OPTIONS:")
print("="*100)
print("To save hourly forecast as CSV:")
print("  hourly_df.to_csv('hourly_forecast_3days.csv', index=False)")
print("\nTo save as JSON for API:")
print("  hourly_df.to_json('hourly_forecast_3days.json', orient='records', date_format='iso')")
print("="*100)

## 14. Hourly Weather Forecast for 3 Days

Generate hourly predictions for the next 3 days (72 hours) using realistic diurnal patterns.

In [ ]:
# Generate hourly forecast for 3 days (72 hours)
start_date = datetime(2025, 11, 27, 0, 0)  # Start from midnight
hours_to_predict = 72  # 3 days

print("="*100)
print("HOURLY WEATHER FORECAST FOR 3 DAYS (72 HOURS)")
print("="*100)
print(f"Starting: {start_date.strftime('%A, %d %B %Y %H:%M')}\n")

hourly_forecast = []

# Get daily predictions first
daily_predictions = {}
for day_offset in range(3):
    current_date = start_date + timedelta(days=day_offset)
    input_df = pd.DataFrame([[current_date.day, current_date.month, current_date.year]], 
                           columns=['day', 'month', 'year'])
    pred = loaded_model.predict(input_df)[0]
    daily_predictions[day_offset] = {
        'tempmax': pred[0],
        'tempmin': pred[1],
        'temp_avg': pred[2],
        'humidity': pred[3],
        'windspeed': pred[4],
        'pressure': pred[5],
        'conditions': int(round(pred[6]))
    }

# Generate hourly data with realistic patterns
for hour in range(hours_to_predict):
    current_time = start_date + timedelta(hours=hour)
    hour_of_day = current_time.hour
    day_offset = hour // 24
    
    daily_pred = daily_predictions[day_offset]
    
    # Temperature pattern: cooler at night (2am), warmest at 2pm
    # Using sinusoidal pattern
    hour_angle = (hour_of_day - 2) * (2 * np.pi / 24)  # 2am is minimum
    temp_factor = (np.sin(hour_angle) + 1) / 2  # 0 to 1
    temp = daily_pred['tempmin'] + (daily_pred['tempmax'] - daily_pred['tempmin']) * temp_factor
    
    # Humidity: inverse to temperature (higher at night)
    humidity_base = daily_pred['humidity']
    humidity_variation = 10  # ±10%
    humidity = humidity_base + humidity_variation * (1 - temp_factor)
    humidity = min(100, max(0, humidity))  # Clamp to 0-100
    
    # Wind speed: slightly higher during day
    wind_factor = 0.7 + 0.3 * temp_factor  # 0.7 to 1.0
    windspeed = daily_pred['windspeed'] * wind_factor + np.random.normal(0, 0.3)
    windspeed = max(0, windspeed)
    
    # Pressure: relatively stable with small variations
    pressure = daily_pred['pressure'] + np.random.normal(0, 0.5)
    
    hourly_forecast.append({
        'datetime': current_time,
        'datetime_str': current_time.strftime('%Y-%m-%d %H:%M'),
        'day': current_time.strftime('%d %b'),
        'hour': hour_of_day,
        'temp': temp,
        'humidity': humidity,
        'windspeed': windspeed,
        'pressure': pressure,
        'conditions': daily_pred['conditions']
    })

# Convert to DataFrame
hourly_df = pd.DataFrame(hourly_forecast)

# Display summary (first 24 hours)
print("Sample: First 24 Hours Forecast")
print("=" * 100)
for i in range(24):
    data = hourly_df.iloc[i]
    if i % 3 == 0:  # Print every 3 hours for readability
        conditions_name = {0: '☀️ Clear', 1: '☁️ Overcast', 2: '⛅ Partly Cloudy', 3: '🌧️ Rain'}[data['conditions']]
        print(f"{data['datetime_str']} | Temp: {data['temp']:.1f}°C | Humidity: {data['humidity']:.1f}% | "
              f"Wind: {data['windspeed']:.1f} m/s | {conditions_name}")

print("\n" + "=" * 100)
print(f"Total hourly predictions: {len(hourly_df)} hours")
print("Full data saved in 'hourly_df' variable")
print("=" * 100)

### Visualize 72-Hour Hourly Forecast

In [ ]:
# Create comprehensive hourly visualization
fig, axes = plt.subplots(4, 1, figsize=(18, 14))

hours_range = range(len(hourly_df))
hour_labels = [hourly_df.iloc[i]['datetime_str'] if i % 6 == 0 else '' for i in hours_range]

# Temperature
ax1 = axes[0]
ax1.plot(hours_range, hourly_df['temp'], linewidth=2, color='#FF6B6B', label='Temperature')
ax1.fill_between(hours_range, hourly_df['temp'], alpha=0.3, color='#FF6B6B')
ax1.set_ylabel('Temperature (°C)', fontsize=12, fontweight='bold')
ax1.set_title('🌡️ 72-Hour Temperature Forecast', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.legend(loc='upper right')
ax1.set_xticks(range(0, len(hourly_df), 6))
ax1.set_xticklabels([hourly_df.iloc[i]['datetime_str'] for i in range(0, len(hourly_df), 6)], 
                     rotation=45, ha='right')

# Add day separators
for day in range(1, 4):
    ax1.axvline(x=day*24, color='gray', linestyle='--', alpha=0.5, linewidth=2)

# Humidity
ax2 = axes[1]
ax2.plot(hours_range, hourly_df['humidity'], linewidth=2, color='#45B7D1', label='Humidity')
ax2.fill_between(hours_range, hourly_df['humidity'], alpha=0.3, color='#45B7D1')
ax2.set_ylabel('Humidity (%)', fontsize=12, fontweight='bold')
ax2.set_title('💧 72-Hour Humidity Forecast', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.legend(loc='upper right')
ax2.set_xticks(range(0, len(hourly_df), 6))
ax2.set_xticklabels([hourly_df.iloc[i]['datetime_str'] for i in range(0, len(hourly_df), 6)], 
                     rotation=45, ha='right')

for day in range(1, 4):
    ax2.axvline(x=day*24, color='gray', linestyle='--', alpha=0.5, linewidth=2)

# Wind Speed
ax3 = axes[2]
ax3.plot(hours_range, hourly_df['windspeed'], linewidth=2, color='#95E1D3', label='Wind Speed')
ax3.fill_between(hours_range, hourly_df['windspeed'], alpha=0.3, color='#95E1D3')
ax3.set_ylabel('Wind Speed (m/s)', fontsize=12, fontweight='bold')
ax3.set_title('💨 72-Hour Wind Speed Forecast', fontsize=14, fontweight='bold')
ax3.grid(True, alpha=0.3)
ax3.legend(loc='upper right')
ax3.set_xticks(range(0, len(hourly_df), 6))
ax3.set_xticklabels([hourly_df.iloc[i]['datetime_str'] for i in range(0, len(hourly_df), 6)], 
                     rotation=45, ha='right')

for day in range(1, 4):
    ax3.axvline(x=day*24, color='gray', linestyle='--', alpha=0.5, linewidth=2)

# Pressure
ax4 = axes[3]
ax4.plot(hours_range, hourly_df['pressure'], linewidth=2, color='#A8E6CF', label='Pressure')
ax4.fill_between(hours_range, hourly_df['pressure'], alpha=0.3, color='#A8E6CF')
ax4.set_ylabel('Pressure (hPa)', fontsize=12, fontweight='bold')
ax4.set_xlabel('Date & Time', fontsize=12, fontweight='bold')
ax4.set_title('⏲️ 72-Hour Pressure Forecast', fontsize=14, fontweight='bold')
ax4.grid(True, alpha=0.3)
ax4.legend(loc='upper right')
ax4.set_xticks(range(0, len(hourly_df), 6))
ax4.set_xticklabels([hourly_df.iloc[i]['datetime_str'] for i in range(0, len(hourly_df), 6)], 
                     rotation=45, ha='right')

for day in range(1, 4):
    ax4.axvline(x=day*24, color='gray', linestyle='--', alpha=0.5, linewidth=2)

plt.tight_layout()
plt.savefig('../outputs/72hour_forecast.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ 72-hour hourly forecast visualization saved as '../outputs/72hour_forecast.png'")
print("\n" + "="*100)
print("EXPORT OPTIONS:")
print("="*100)
print("To save hourly forecast as CSV:")
print("  hourly_df.to_csv('hourly_forecast_3days.csv', index=False)")
print("\nTo save as JSON for API:")
print("  hourly_df.to_json('hourly_forecast_3days.json', orient='records', date_format='iso')")
print("="*100)